In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
import pandas as pd
df = pd.read_excel("E Commerce Dataset.xlsx", sheet_name="E Comm")

print(df.shape)
df.head()
df.info()


In [ ]:
print("Missing values per column:")
df.isnull().sum()


In [ ]:
print("Missing value percentage:")
(df.isnull().mean() * 100).round(2)


In [ ]:
df['Churn'].value_counts(normalize=True) * 100


In [ ]:
df.describe()


In [ ]:
numeric_cols = ['Tenure','WarehouseToHome','HourSpendOnApp','OrderCount','CashbackAmount']

df[numeric_cols].hist(figsize=(12,8))
plt.show()


In [ ]:
categorical_cols = ['PreferredLoginDevice','PreferredPaymentMode','Gender',
                    'PreferedOrderCat','MaritalStatus']

for col in categorical_cols:
    print(f"\nValue counts for {col}:")
    print(df[col].value_counts())


In [ ]:
key_cols = ['Tenure','SatisfactionScore','Complain','OrderCount','CashbackAmount']

for col in key_cols:
    print(f"\nMean {col} by Churn:")
    print(df.groupby('Churn')[col].mean())


In [ ]:
target = "Churn"

numeric_features = [
    "Tenure",
    "WarehouseToHome",
    "HourSpendOnApp",
    "SatisfactionScore",
    "OrderAmountHikeFromlastYear",
    "CouponUsed",
    "OrderCount",
    "DaySinceLastOrder",
    "CashbackAmount"
]

categorical_features = [
    "PreferredLoginDevice",
    "PreferredPaymentMode",
    "Gender",
    "PreferedOrderCat",
    "MaritalStatus"
]


In [ ]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [ ]:
X = df[numeric_features + categorical_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
baseline_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=200, class_weight='balanced'))
])


In [ ]:
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


In [ ]:
baseline_model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=300, class_weight='balanced'))
])

baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)
y_proba = baseline_model.predict_proba(X_test)[:,1]

print("AUC:", roc_auc_score(y_test, y_proba))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
X_train_nn = preprocessor.fit_transform(X_train)
X_test_nn = preprocessor.transform(X_test)


In [ ]:
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier(
    hidden_layer_sizes=(16,),
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=42
)

nn_model.fit(X_train_nn, y_train)


In [ ]:
nn_pred_proba = nn_model.predict_proba(X_test_nn)[:, 1]

from sklearn.metrics import roc_auc_score

print("Neural Network AUC:", roc_auc_score(y_test, nn_pred_proba))


In [ ]:
baseline_auc = roc_auc_score(y_test, y_proba)
nn_auc = roc_auc_score(y_test, nn_pred_proba)

comparison = pd.DataFrame({
    "Model": ["Logistic Regression", "Neural Network (MLP)"],
    "AUC Score": [baseline_auc, nn_auc]
})

comparison
